##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image classification

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial shows how to classify cats or dogs from images. It builds an image classifier using a `tf.keras.Sequential` model and load data using `tf.keras.preprocessing.image.ImageDataGenerator`. You will get some practical experience and develop intuition for the following concepts:

* Building _data input pipelines_ using the `tf.keras.preprocessing.image.ImageDataGenerator` class to efficiently work with data on disk to use with the model.
* _Overfitting_ —How to identify and prevent it.
* _Data augmentation_ and _dropout_ —Key techniques to fight overfitting in computer vision tasks to incorporate into the data pipeline and image classifier model.

This tutorial follows a basic machine learning workflow:

1. Examine and understand data
2. Build an input pipeline
3. Build the model
4. Train the model
5. Test the model
6. Improve the model and repeat the process

## Import packages

Let's start by importing the required packages. The `os` package is used to read files and directory structure, NumPy is used to convert python list to numpy array and to perform required matrix operations and `matplotlib.pyplot` to plot the graph and display images in the training and validation data.

Import Tensorflow and the Keras classes needed to construct our model.

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import keras

## Load data

Begin by downloading the dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

After extracting its contents, assign variables with the proper file path for the training and validation set.

In [ ]:
import os

PATH = '/content/drive/My Drive/data/final_dataset_balanced/'
train_dir = os.path.join(PATH, 'train')
test_dir = os.path.join(PATH, 'test')

### Understand the data

Let's look at how many cats and dogs images are in the training and validation directory:

In [ ]:
num_tr = len(os.listdir(train_dir))
num_val = len(os.listdir(test_dir))

In [ ]:
print(os.listdir(train_dir))

In [ ]:
num_tr

For convenience, set up variables to use while pre-processing the dataset and training the network.

In [ ]:
batch_size = 20
epochs = 15
IMG_HEIGHT = 224
IMG_WIDTH = 224

## Data preparation

Format the images into appropriately pre-processed floating point tensors before feeding to the network:

1. Read images from the disk.
2. Decode contents of these images and convert it into proper grid format as per their RGB content.
3. Convert them into floating point tensors.
4. Rescale the tensors from values between 0 and 255 to values between 0 and 1, as neural networks prefer to deal with small input values.

Fortunately, all these tasks can be done with the `ImageDataGenerator` class provided by `tf.keras`. It can read images from disk and preprocess them into proper tensors. It will also set up generators that convert these images into batches of tensors—helpful when training the network.

In [ ]:
image_gen_train = ImageDataGenerator(
                    rescale=1./255,
                    validation_split=0.3,
                    horizontal_flip=True,
                    vertical_flip=True,
                    rotation_range=90
                    )

In [ ]:
train_image_generator = image_gen_train # Generator for our training data
validation_image_generator = image_gen_train # Generator for our validation data

In [ ]:
image_gen_test = ImageDataGenerator(
                    rescale = 1./255)

In [ ]:
test_image_generator = image_gen_test

After defining the generators for training and validation images, the `flow_from_directory` method load images from the disk, applies rescaling, and resizes the images into the required dimensions.

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size, #directory containing various tangram
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH), # all images will be resized to 150, 150 when it is loaded
                                                           class_mode='categorical',
                                                           subset='training') 

In [ ]:
val_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=train_dir,
                                                              shuffle=True,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='categorical',
                                                              subset='validation')

### Visualize training images

Visualize the training images by extracting a batch of images from the training generator—which are 128 images in this example—then plot five of them with `matplotlib`.

In [ ]:
sample_training_images, _ = next(train_data_gen)

The `next` function returns a batch from the dataset. The return value of `next` function is in form of `(x_train, y_train)` where x_train is training features and y_train, its labels. Discard the labels to only visualize the training images.

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(2, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_training_images[:10])

## Create the model

We are using transfer learning for this application and we chose MobilNetV2 for its light weightness. We will just replace the top layer by a custom layer to match our requirements of 12 different classes to be detected.

In [ ]:
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.trainable = False
base_model.summary()


In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [ ]:
prediction_layer = tf.keras.layers.Dense(12, activation='softmax')

In [ ]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [ ]:
def scheduler(epoch, lr):
  if epoch < 3:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

## Compile the model

We choose the rmsprop optimizer and categorical_crossentropy loss function. To view training and validation accuracy for each training epoch, pass the metrics argument.

In [ ]:
lr = 0.001
model.compile(optimizer='Nadam',
              loss="categorical_crossentropy",
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', verbose=1, patience=5)

In [ ]:
initial_epochs = 10
validation_steps=20

loss0,accuracy0 = model.evaluate(val_data_gen, steps = validation_steps)

## Train the model

In [ ]:
history = model.fit(train_data_gen,
                    epochs=45,
                    callbacks=[callback_lr, callback_es],
                    validation_data=val_data_gen)

In [ ]:
from tensorflow import keras


### Sauvegarde du modèle

In [ ]:


model.save("tangram_jason_mobilenet_final_10082020_2.h5")



### Visualize training results

Now visualize the results after training the network.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(12)

plt.figure(figsize=(7, 7))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
acc[-1], val_acc[-1], loss[-1], val_loss[-1]

## Definition for prediction

In [ ]:
import time
import cv2
import os
import numpy as np

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model = load_model("/content/drive/My Drive/saved_model/tangram_inceptionV3_full.h5")

In [ ]:
test_data_gen = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=test_dir,
                                                              shuffle=False,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='categorical')

In [ ]:
test_data_gen.reset()

In [ ]:
pred=model.predict_generator(test_data_gen,verbose=1,steps=960/batch_size)



In [ ]:
pred[1][0]

In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)

labels = (train_data_gen.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
result = {}

for i in range(len(pred[25])):
  result[labels[i]]=pred[25][i]



In [ ]:
import pandas as pd

In [ ]:
filenames=test_data_gen.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})

In [ ]:
real_pred = []

for filename in filenames:
  real_pred.append(filename.split('/')[0])

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(real_pred, predictions, labels=["bateau", "bol", "chat", "coeur", "cygne", "lapin", "maison", "marteau", "montagne", "pont", "renard", "tortue"])

print(cm)

### Classification report

In [ ]:
from sklearn.metrics import classification_report 

print(classification_report(real_pred, predictions))


### Accuracy calculation

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(real_pred, predictions)

In [ ]:
results

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt


In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(7,7))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(cm=cm, classes=train_data_gen.class_indices, title='Confusion Matrix')

In [ ]:
def result_image(path):
  start_time = time.time()
  
  ### Preprocessing
  start_time_prepro = time.time()
  img = cv2.imread(path)
  height, width, channels = img.shape
  width_cutoff = width // 2
  s1 = img[:, :width_cutoff]
  s2 = img[:, width_cutoff:]
  s1_up = tf.image.resize(s1/255, (224,224), preserve_aspect_ratio=False)
  s1_final = tf.expand_dims(s1_up, axis=0)
  s2_up = tf.image.resize(s2/255, (224,224), preserve_aspect_ratio=False)
  s2_final = tf.expand_dims(s2_up, axis=0)
  end_time_prepro = time.time()
  print("Preprocessing time:", end_time_prepro - start_time_prepro)

  ### Prediction
  start_time_pred = time.time()
  labels = {0: 'bateau', 1: 'bol', 2: 'chat', 3: 'coeur', 4: 'cygne', 5: 'lapin', 6: 'maison', 7: 'marteau', 8: 'montagne', 9: 'pont', 10: 'renard', 11: 'tortue'}
  result_1 = model.predict(s1_final)
  result_dict_1 = {}
  result_2 = model.predict(s2_final)
  result_dict_2 = {}
  for i in range(len(result_1[0])):
    result_dict_1[labels[i]]=result_1[0][i]
    result_dict_2[labels[i]]=result_2[0][i]
  end_time_pred = time.time()
  print("Prediction time:", end_time_pred - start_time_pred)
  
  end_time = time.time()
  total_fps = 1/(end_time-start_time)
  print("Total time:",end_time-start_time)
  print("FPS:",total_fps)
  return result_dict_1, result_dict_2

In [ ]:
path_img = "/content/image1375.jpg"

In [ ]:
result_image(path_img)